In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np

In [2]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
data


,No.,Time,Source,Destination,Protocol,Length,Topic,Message Identifier,Source Port,Destination Port,Info
0,284,34.387541,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,45295,1883,Connect Command
1,285,34.389140,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,53853,1883,Connect Command
2,287,34.389833,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,51817,1883,Connect Command
3,288,34.390308,192.168.15.4,192.168.15.3,MQTT,90,NaN,NaN,50551,1883,Connect Command
4,289,34.390400,192.168.15.3,192.168.15.4,MQTT,70,NaN,NaN,1883,45295,Connect Ack
...,...,...,...,...,...,...,...,...,...,...,...
1015,2718,134.026118,192.168.15.3,192.168.15.4,MQTT,70,NaN,100.0,1883,53853,Publish Ack (id=100)
1016,2720,134.026678,192.168.15.3,192.168.15.4,MQTT,70,NaN,100.0,1883,45295,Publish Ack (id=100)
1017,2722,134.026678,192.168.15.3,192.168.15.4,MQTT,70,NaN,100.0,1883,41481,Publish Ack (id=100)
1018,2724,134.026711,192.168.15.3,192.168.15.4,MQTT,70,NaN,100.0,1883,51817,Publish Ack (id=100)


In [3]:
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports:  
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [4]:
data = {}
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT


## Medidas de Tendência Central Por Device

In [5]:
i=1
data_mean = []
data_median = []
data_std = []
data_min = []
data_max = []

for port in ports:
  data_mean.append(data[port]["RTT"].mean())
  data_median.append(data[port]["RTT"].mean())
  data_std.append(data[port]["RTT"].mean())
  data_min.append(data[port]["RTT"].mean())
  data_max.append(data[port]["RTT"].mean())

  print("(Device-{})Mean Round Trip Time = {}".format(i ,data[port]["RTT"].mean()))
  print("(Device-{})Median Round Trip Time = {}".format(i ,data[port]["RTT"].median()))
  print("(Device-{})Standard deviation  Round Trip Time = {}".format(i ,data[port]["RTT"].std()))
  print("(Device-{})Min Round Trip Time = {}".format(i ,data[port]["RTT"].min()))
  print("(Device-{})Max Round Trip Time = {}".format(i ,data[port]["RTT"].max()))
  print("---------------------------------------------------------------")

  i+=1

(Device-1)Mean Round Trip Time = 2.8735761799997306
(Device-1)Median Round Trip Time = 2.80626749999513
(Device-1)Standard deviation  Round Trip Time = 0.7397013895275015
(Device-1)Min Round Trip Time = 1.5871860000231663
(Device-1)Max Round Trip Time = 9.174368999993021
---------------------------------------------------------------
(Device-2)Mean Round Trip Time = 2.91897285999994
(Device-2)Median Round Trip Time = 2.8107860000083917
(Device-2)Standard deviation  Round Trip Time = 0.7377753990284032
(Device-2)Min Round Trip Time = 1.5128349999997681
(Device-2)Max Round Trip Time = 8.768222999975706
---------------------------------------------------------------
(Device-3)Mean Round Trip Time = 2.834072150000111
(Device-3)Median Round Trip Time = 2.8189165000007677
(Device-3)Standard deviation  Round Trip Time = 0.5373536953068079
(Device-3)Min Round Trip Time = 1.6353270000024622
(Device-3)Max Round Trip Time = 6.927015999991681
-------------------------------------------------------

## Medidas de Tendência Central Geral

In [6]:
from statistics import mean 

print("Mean Round Trip Time = {}".format(mean(data_mean)))
print("Median Round Trip Time = {}".format(mean(data_median)))
print("Standard deviation  Round Trip Time = {}".format(mean(data_std)))
print("Min Round Trip Time = {}".format(mean(data_min)))
print("Max Round Trip Time = {}".format(mean(data_max)))


Mean Round Trip Time = 2.8652622499999154
Median Round Trip Time = 2.8652622499999154
Standard deviation  Round Trip Time = 2.8652622499999154
Min Round Trip Time = 2.8652622499999154
Max Round Trip Time = 2.8652622499999154
